In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI

# Aufgabe 08: Minimax mit Alpha-Beta-Pruning, Memoisierung, Progressive Deepening und Singular Value Extension

Dieses Notebook erweitert den Minimax-Algorithmus um die Singular Value Extension.

In [ ]:
import chess
from typing import Any, Callable


class Exercise08AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization and progressive deepening and the quiescence search"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, Any] = {}
        self.real_depth: int = 0

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache.clear()
        self.real_depth = 0

In [ ]:
def debug_minimax(evaluate_minimax: Callable):
    """Prints the Minimax Game Tree."""

    def minimax_debug(
        self,
        minimax: Callable,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        depth: int = 0,
        alpha: int = -Exercise08AI.LIMIT,
        beta: int = Exercise08AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        color = "white" if board.turn else "black"
        ws = (depth + 1) * "    "
        key = Exercise04AI.get_key(board)
        if key in self.cache:
            print(
                f"depth: {depth} {ws} mm> {color}; α: {alpha}; β: {beta}; limit: {limit}"
            )  # memoization
        else:
            print(
                f"depth: {depth} {ws} --> {color}; α: {alpha}; β: {beta}; limit: {limit}"
            )
        evaluation, best_move = evaluate_minimax(
            self, minimax, board, current_evaluation, limit, depth, alpha, beta
        )
        print(
            f"depth: {depth} {ws} <-- {color}; ev: {evaluation}; best_move: {best_move}"
        )
        return evaluation, best_move

    return minimax_debug

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def is_quiet_move(self, board: chess.Board, move: chess.Move) -> bool:
        """Checks if the next move was an promotion, capture or check move. Pushes the given move on the board."""
        if move.promotion:
            board.push(move)
            return False
        if board.piece_type_at(move.to_square):
            board.push(move)
            return False
        board.push(move)
        if board.is_check():
            return False
        return True

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def minimax_early_abort(
        self, board: chess.Board, depth: int, limit: int, current_evaluation: int
    ) -> int | None:
        """Returns an evaluation iff the minimax has an early exit condition. Returns None otherwise."""
        if (is_checkmate := board.is_checkmate()) and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - depth
            return evaluation

        elif is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + depth
            return evaluation

        elif (
            board.is_insufficient_material()
            or not board.legal_moves
            or board.is_fifty_moves()
        ):
            # Game is a draw
            return 0

        # Recursion abort case
        if depth == limit:
            return current_evaluation

        return None

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def maxValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        depth: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm"""
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, limit, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += depth
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # White to play (positive numbers are good)
        for i, move in enumerate(board.legal_moves):
            key = Exercise04AI.get_key(board)
            old_eval = self.cache.get(key, (None, i))[1]
            heapq.heappush(moves, (-old_eval, i, move))
        maxEvaluation = alpha
        while moves:
            move = heapq.heappop(moves)[2]
            if depth + 1 < limit:
                new_limit = limit
                board.push(move)
            elif self.is_quiet_move(board, move):
                new_limit = limit
            else:
                new_limit = limit + 1
            evaluation, _, = self.evaluate_minimax(
                self.minValue,
                board,
                self.evaluate(board, current_evaluation),
                new_limit,
                depth + 1,
                maxEvaluation,
                beta,
            )
            board.pop()
            if evaluation >= beta:
                return evaluation, move
            if evaluation > maxEvaluation:
                best_move = move
                maxEvaluation = evaluation
        return maxEvaluation, best_move

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def minValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        depth: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm"""
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, limit, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += depth
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # Black to play (negative numbers are good)
        for i, move in enumerate(board.legal_moves):
            key = Exercise04AI.get_key(board)
            old_eval = self.cache.get(key, (None, i))[1]
            heapq.heappush(moves, (old_eval, i, move))
        minEvaluation = beta
        while moves:
            move = heapq.heappop(moves)[2]
            if depth + 1 < limit:
                new_limit = limit
                board.push(move)
            elif self.is_quiet_move(board, move):
                new_limit = limit
            else:
                new_limit = limit + 1
            evaluation, _ = self.evaluate_minimax(
                self.maxValue,
                board,
                self.evaluate(board, current_evaluation),
                new_limit,
                depth + 1,
                alpha,
                minEvaluation,
            )
            board.pop()
            if evaluation <= alpha:
                return evaluation, move
            if evaluation < minEvaluation:
                best_move = move
                minEvaluation = evaluation
        return minEvaluation, best_move

In [ ]:
import heapq


class Exercise08AI(Exercise08AI):  # type: ignore
    # @debug_minimax
    def evaluate_minimax(
        self,
        minimax: Callable,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        depth: int = 0,
        alpha: int = -Exercise08AI.LIMIT,
        beta: int = Exercise08AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm"""
        key = Exercise04AI.get_key(board)
        self.stats[-1]["cache_tries"] += 1
        self.stats[-1]["max_depth"] = max(depth, self.stats[-1]["max_depth"])

        if key in self.cache:
            self.stats[-1]["cache_hits"] += 1
            return self.get_from_cache(
                minimax,
                key,
                board,
                current_evaluation,
                limit,
                depth,
                alpha,
                beta,
            )
        result = minimax(board, current_evaluation, limit, depth, alpha, beta)
        self.store_in_cache(key, result, limit, depth, alpha, beta)
        return result

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def store_in_cache(
        self, key: tuple, result: tuple, limit, depth, alpha: int, beta: int
    ) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation < alpha:
            self.cache[key] = ("≤", evaluation, move, limit - depth)
        elif evaluation <= beta:
            self.cache[key] = ("=", evaluation, move, limit - depth)
        else:
            self.cache[key] = ("≥", evaluation, move, limit - depth)

    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        current_eval: int,
        limit: int,
        depth: int,
        alpha: int,
        beta: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move, eval_depth = self.cache[key]
        if limit > eval_depth:
            result = minimax(board, current_eval, limit, depth, alpha, beta)
            self.store_in_cache(key, result, limit, depth, alpha, beta)
            return result
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(board, current_eval, limit, depth, alpha, evaluation)
                self.store_in_cache(key, result, limit, depth, alpha, evaluation)
                return result
            else:
                result = minimax(board, current_eval, limit, depth, alpha, beta)
                self.store_in_cache(key, result, limit, depth, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(board, current_eval, limit, depth, alpha, beta)
                self.store_in_cache(key, result, limit, depth, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(board, current_eval, limit, depth, evaluation, beta)
                self.store_in_cache(key, result, limit, depth, evaluation, beta)
                return result
            else:
                return evaluation, move

In [ ]:
import sys
import gc


class Exercise08AI(Exercise08AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        self.stats[-1]["cache_hits"] = 0
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["leaf_ctr"] = 0
        self.stats[-1]["max_depth"] = -1
        self.stats[-1]["avg_depth"] = -1
        self.stats[-1]["depth_sum"] = 0

        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for limit in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            if board.turn:
                future_evaluation, best_move = self.evaluate_minimax(
                    self.maxValue, board, current_evaluation, limit
                )
            else:
                future_evaluation, best_move = self.evaluate_minimax(
                    self.minValue, board, current_evaluation, limit
                )

        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(
            board, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation
        self.stats[-1]["avg_depth"] = self.stats[-1]["depth_sum"] / (
            self.stats[-1]["leaf_ctr"] or 1
        )
        del self.stats[-1]["depth_sum"]
        del self.stats[-1]["leaf_ctr"]
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        if board.is_irreversible(best_move):
            del self.cache
            gc.collect()
            self.stats[-1]["cache_cleared"] = True
            self.cache: dict[tuple, Any] = {}
        else:
            self.stats[-1]["cache_cleared"] = False
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise03AI as Exercise03AITests
import Exercise04AI as Exercise04AITests

In [ ]:
# Create player and board
unit_test_player = Exercise08AI(player_name="Ex08AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
def test_minimax(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    unit_test_player.stats[-1]["leaf_ctr"] = 0
    unit_test_player.stats[-1]["max_depth"] = 0
    unit_test_player.stats[-1]["depth_sum"] = 0
    f = unit_test_player.maxValue if board.turn else unit_test_player.minValue
    mm_evaluation, mm_move = unit_test_player.evaluate_minimax(
        f, board, current_evaluation=1240, limit=unit_test_player.DEPTH
    )
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == "d4c3", "Minimax move does not match expected value!"
    assert unit_test_player.cache != {}, "Cache is empty!"
    print(f"Elements in cache: {len(unit_test_player.cache)}")

In [ ]:
test_minimax(unit_test_player, board)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03AITests.test_next_move(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.